In [1]:
import xarray as xr
import numpy as np
import scipy as sp
import seastar as ss
import warnings
import os
from _version import __version__

In [2]:
save_folder=os.path.join('D:\data\SEASTAR\SEASTARex\Data\L1b','')
version = 'v20230215'
days = ['17','22','25','26']
flights = ['202205' + d for d in days]
file_paths = ss.utils.readers._set_file_paths()
vars_to_keep = ['LatImage', 'LonImage', 'IncidenceAngleImage', 'LookDirection', 'SquintImage', 'CentralFreq',
               'OrbitHeadingImage']

Device name = LIVPCZG-10
Setting local paths...


In [3]:
for flight in flights:
    oscar_path = os.path.join(file_paths['oscar_path'], version, flight, 'matlab-pre-processed','')
    file_time_triplets = ss.oscar.tools.find_file_triplets(oscar_path)
    print('OSCAR L1 processing - data version ' + version + '\n-------------------')
    print('File path :', oscar_path + '\n-------------------')
    for i in range(len(file_time_triplets)):
        print('File index :', i, ',', 'Aquisition time :', file_time_triplets[i][0], ',', 'File list indices :', file_time_triplets[i][1])
    DAR_tracks = ss.utils.readers._read_DAR_config(flight)
    ds_L1b = dict()
    for track in DAR_tracks.keys():
        ds_ml = dict()
        file_index = DAR_tracks[track]
        ds_L1a = ss.oscar.tools.load_OSCAR_data(oscar_path, file_time_triplets[file_index][1])
        ds_L1a_dict_index = list(ds_L1a.keys())
        antenna_ident = ss.oscar.tools.identify_antenna_location_from_filename(
                            oscar_path,
                            file_time_triplets[file_index][1])
        antenna_dict = {list(ds_L1a.keys())[i]: antenna_ident[i] for i in range(len(antenna_ident))}
        print('Processing file index',file_index,'...')
        # L1b processing - loop through beams in L1a dataset dict
        #--------------------------------------------------------
        for dict_index in ds_L1a_dict_index:
            ds_L1a[dict_index] = ss.oscar.level1.replace_dummy_values(
                                     ds_L1a[dict_index],
                                     dummy_val=int(ds_L1a[dict_index].Dummy.data))
            ds_ml[dict_index] = ss.oscar.level1.compute_multilooking_Master_Slave(
                                    ds_L1a[dict_index],
                                    window=7)
            #ds_ml[dict_index]['Polarization'] = ss.oscar.level1.check_antenna_polarization(ds_L1a[dict_index])
            ds_ml[dict_index]['Polarization'] = xr.DataArray(data=np.full(ds_L1a[dict_index].IncidenceAngleImage.shape,
                                                                  ss.oscar.level1.check_antenna_polarization(
                                                                      ds_L1a[dict_index]).data),
                                             coords=ds_L1a[dict_index].IncidenceAngleImage.coords,
                                             dims=ds_L1a[dict_index].IncidenceAngleImage.dims)
            ss.oscar.level1.check_antenna_polarization(ds_L1a[dict_index])
            ds_ml[dict_index]['AntennaAzimuthImage'] =ss.oscar.level1.compute_antenna_azimuth_direction(
                                                        ds_L1a[dict_index],
                                                        antenna=antenna_ident[ds_L1a_dict_index.index(dict_index)])
            ds_ml[dict_index]['TimeLag'] = ss.oscar.level1.compute_time_lag_Master_Slave(
                                               ds_L1a[dict_index],
                                               options='from_SAR_time')
            ds_ml[dict_index].attrs['Title'] = ds_L1a[dict_index].Title
            ds_ml[dict_index]['TrackTime'] = ss.oscar.level1.track_title_to_datetime(ds_ml[dict_index].Title)
            ds_ml[dict_index][vars_to_keep] = ds_L1a[dict_index][vars_to_keep]
        ds_ml = ss.oscar.level1.fill_missing_variables(ds_ml, antenna_ident)
            # Build L1b dataset
        ds_L1b[track] = ss.oscar.level1.merge_beams(ds_ml, antenna_ident)
        ds_L1b[track].attrs['Codebase'] = 'seastar_project'
        ds_L1b[track].attrs['Repository'] = 'https://github.com/NOC-EO/seastar_project'
        ds_L1b[track].attrs['Code_version'] = __version__
        ds_L1b[track].attrs['Data_version'] = version
        print('Saving to disk...')
        ds_L1b[track].to_netcdf(os.path.join(save_folder, flight + '_' + track +'_OSCAR_L1b.nc'))

OSCAR L1 processing - data version v20230215
-------------------
File path : D:\data\SEASTAR\SEASTARex\Data\Metasensing\OSCAR\v20230215\20220517\matlab-pre-processed\
-------------------
File index : 0 , Aquisition time : 20220517T090013 , File list indices : [0, 1, 2]
File index : 1 , Aquisition time : 20220517T090809 , File list indices : [3, 4, 5]
File index : 2 , Aquisition time : 20220517T091519 , File list indices : [6, 7, 8]
File index : 3 , Aquisition time : 20220517T093239 , File list indices : [9, 10, 11]
File index : 4 , Aquisition time : 20220517T093818 , File list indices : [12, 13, 14]
File index : 5 , Aquisition time : 20220517T094339 , File list indices : [15, 16, 17]
File index : 6 , Aquisition time : 20220517T101737 , File list indices : [18, 19, 20]
Processing file index 0 ...


KeyError: 'CentralWavenumber'